In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import pymc as pm
import numpy as np
from constants import stablecoins
from utils import create_graph #, plot_degree_distribution, edge_to_remove, girvan_newman

ModuleNotFoundError: No module named 'networkx'

### Load Data

In [ ]:
df = pd.read_csv("all_coins_data.csv")

: 

In [ ]:
#df = read_data("all_coins_data.csv")
nodes = list(set(df.coin))#Get unique list of coins
volatile_coins = [n for n in nodes if nodes not in stablecoins]
df["stablecoin"] = [1 if c in stablecoins else 0 for c in df["coin"]]
df.to_csv("df_app_data.csv", sep=";", index = False)
df.head()

: 

In [ ]:
df.head()

: 

#### Return Analysis

In [ ]:
df_return_wide = df[["time", "close", "coin"]].pivot(index = "time",columns="coin", values = "close").pct_change()
df_return_wide.head()

: 

In [ ]:
df["stablecoin"] = [1 if c in stablecoins else 0 for c in df["coin"]]
df_long = pd.melt(df, id_vars=["coin","time", "stablecoin"], value_vars=["high","low","open","volumefrom","volumeto","close","daily_return_pct","intraday_return","daily_log_return"], ignore_index=False).set_index("time")
df_long.head()

: 

In [ ]:
#Stable coins
fig = px.line(df_long[(df_long.variable == "close") & 
(df_long.stablecoin == 0)].reset_index()[["time","coin","value"]], 
x = "time", y = "value", color = 'coin')
fig.show()


: 

In [ ]:
#Volatile coins
fig = px.line(df_long[(df_long.variable == "close") & 
(df_long.stablecoin == 1)].reset_index()[["time","coin","value"]], 
x = "time", y = "value", color = 'coin')
fig.show()


: 

### Correlation Matrix

In [ ]:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(df_long[df_long["variable"]=="daily_log_return"].reset_index().drop(["variable", "stablecoin"], axis = 1).pivot(index = "time",columns="coin").corr(), 
                      vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

: 

In [ ]:
corr_df = df_long[df_long["variable"]=="daily_log_return"].drop(["stablecoin", "variable"], axis = 1).pivot(columns = "coin", values="value").corr()
corr_df = corr_df.unstack(level = 0).to_frame().rename(columns = {0:"corr"})
corr_df = corr_df[corr_df["corr"]!=1].reset_index(level=0).rename(columns={"coin":"cointo"}).reset_index().rename(columns = {"coin":"coinfrom"})

: 

In [ ]:
corr_df

: 

In [ ]:
a4_dims = (11.7, 8.27)
fig, ax = plt.subplots(figsize=(a4_dims))
sns.histplot(corr_df, x = "corr", bins = 100)
corr_df["corr"].describe()

: 